In [8]:
import pandas as pd
import import_ipynb
import initialise_content_and_finder
from initialise_content_and_finder import *


#Rule 1 finds high-risk promotions on a whole page with terms lists 1(in), 2(in), 3(exclusions)
def rule_1(content, offer_lan, high_risk_words, exclusions):
    y = find_terms_in(offer_lan, split_by_whole_page(content), terms_list_name = 'offers_in')
    y = find_terms_in(high_risk_words, y, terms_list_name = 'high_risk_words_in')
    if exclusions != []:
        y = find_terms_notin(exclusions, y, terms_list_name = 'exclusions')
    rule_name = 'Rule 1: High-risk promotions on page'
    y['rule'] = rule_name
    return y, rule_name

#Rule 2 finds high-risk phrases in a sentence on a page.
def rule_2(content, high_risk_words):
    y = find_terms_in(high_risk_words, split_by_sentence(content), terms_list_name = 'high_risk_words_in')
    rule_name = 'Rule 2: High-risk words in sentences'
    y['rule'] = rule_name
    return y, rule_name

#Rule 3 finds a missing disclaimer for an offer.
def rule_3(content, offer_lan, disclaimers):
    y = find_terms_in(offer_lan, split_by_whole_page(content), terms_list_name = 'offers_in')
    y = find_terms_notin(disclaimers, y, terms_list_name = 'missing_disclaimer')
    rule_name = 'Rule 3: Missing sign up disclaimer'
    y['rule'] = rule_name
    return y, rule_name

#Rule 4 finds a missing disclaimer on any page.
def rule_4(content, disclaimers):
    y = find_terms_notin(disclaimers, split_by_whole_page(content), terms_list_name = 'missing_disclaimer')
    rule_name = 'Rule 4: Missing legal disclaimer'
    y['rule'] = rule_name
    return y, rule_name
